In [2]:
%load_ext autoreload
%autoreload

from os import getcwd
from os.path import join, abspath, pardir, relpath, exists

import requests

from dataclasses import dataclass, field

import pandas as pd
import numpy as np
from numpy import matrixlib as npmat
import networkx as nx
from typing import Union
import pulp as p
from itertools import combinations
from typing import List, Tuple
from enum import Enum
from scipy.stats import kendalltau, spearmanr

from IPython.display import IFrame
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
pd.set_option('display.max_columns', None)
parent_dir = abspath(join(getcwd(), pardir))
data_dir = join(parent_dir, 'data')
data_file = join(data_dir, 'products1.csv')
parent_dir

'/Users/mohammadzainabbas/Masters/CS/Decision-Modelling/nutri-score'

In [4]:
df = pd.read_csv(data_file)
df.head()

,id,name,image_url,brand,categories,energy,energy_points,energy_value,fiber,fiber_points,fiber_value,fruits_vegetables_nuts_colza_walnut_olive_oils,fruits_vegetables_nuts_colza_walnut_olive_oils_points,fruits_vegetables_nuts_colza_walnut_olive_oils_value,grade,is_beverage,is_cheese,is_fat,is_water,negative_points,positive_points,proteins,proteins_points,proteins_value,saturated_fat,saturated_fat_points,saturated_fat_ratio,saturated_fat_ratio_points,saturated_fat_ratio_value,saturated_fat_value,score,sodium,sodium_points,sodium_value,sugars,sugars_points,sugars_value,carbohydrates,carbohydrates_100g,carbohydrates_unit,carbohydrates_value,energy-kcal,energy-kcal_100g,energy-kcal_unit,energy-kcal_value,energy-kj,energy-kj_100g,energy-kj_unit,energy-kj_value,energy_100g,energy_unit,fat,fat_100g,fat_unit,fat_value,fiber_100g,fiber_unit,fruits-vegetables-nuts-estimate-from-ingredients_100g,fruits-vegetables-nuts-estimate-from-ingredients_serving,nova-group,nova-group_100g,nova-group_serving,nutrition-score-fr,nutrition-score-fr_100g,proteins_100g,proteins_unit,salt,salt_100g,salt_unit,salt_value,saturated-fat,saturated-fat_100g,saturated-fat_unit,saturated-fat_value,sodium_100g,sodium_unit,sugars_100g,sugars_unit,energy-kcal_value_computed,energy-kj_value_computed,carbohydrates_serving,energy-kcal_serving,energy_serving,fat_serving,fiber_serving,proteins_serving,salt_serving,saturated-fat_serving,sodium_serving,sugars_serving,carbon-footprint-from-known-ingredients_100g,carbon-footprint-from-known-ingredients_product,carbon-footprint-from-known-ingredients_serving,energy-kj_serving
0,3256220881111,Ananas en morceaux au jus d'ananas Bio,https://images.openfoodfacts.org/images/produc...,"U Bio,U","Aliments et boissons à base de végétaux, Alime...",271,0,271,0.7,0,0.7,50.0,1,50.0,b,0,0,0,0,3,1,0.60,0,0.60,0.1000,0,20.000000,2,20.0,0.1,2,0.004000,0,0.004000,15.00,3,15.00,15.00,15.00,g,15.00,64.0,64.0,kcal,64.0,271.0,271.0,kJ,271.0,271,kJ,0.500,0.500,g,0.500,0.7,g,50.0,50.0,1.0,1.0,1.0,2,2,0.60,g,0.01000,0.01000,g,0.01000,0.1000,0.1000,g,0.1000,0.004000,g,15.00,g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5400141049657,Oignons doux 1kg,https://images.openfoodfacts.org/images/produc...,Boni,"Aliments et boissons à base de végétaux,Alimen...",159,0,159,2.8,2,2.8,100.0,5,100.0,a,0,0,0,0,1,7,1.25,0,1.25,0.0443,0,7.572650,0,7.6,0.0,-6,0.039312,0,0.039312,5.35,1,5.35,7.37,7.37,g,7.37,40.2,40.2,kcal,40.2,159.0,159.0,kJ,159.0,159,kJ,0.585,0.585,g,0.585,2.8,g,100.0,100.0,1.0,1.0,1.0,-6,-6,1.25,g,0.09828,0.09828,g,0.09828,0.0443,0.0443,g,0.0443,0.039312,g,5.35,g,45.345,190.585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3487400004437,Blé naturel,https://images.openfoodfacts.org/images/produc...,Ebly,"Aliments et boissons à base de végétaux, Alime...",565,1,135,2.2,2,2.2,0.0,0,0.0,a,0,0,0,0,1,4,4.70,2,4.70,0.1000,0,20.000000,2,20.0,0.1,-3,0.004000,0,0.004000,0.50,0,0.50,27.00,27.00,g,27.00,135.0,135.0,kcal,135.0,NaN,NaN,NaN,NaN,565,kcal,0.500,0.500,g,0.500,2.2,g,0.0,0.0,1.0,1.0,1.0,-3,-3,4.70,g,0.01000,0.01000,g,0.01000,0.1000,0.1000,g,0.1000,0.004000,g,0.50,g,NaN,NaN,27.0,135.0,565.0,0.5,2.2,4.7,0.01,0.1,0.004,0.5,NaN,NaN,NaN,NaN
3,4047247192743,Cashewkerne,https://images.openfoodfacts.org/images/produc...,Trader Joe‘s,"Pflanzliche Lebensmittel und Getränke,Pflanzli...",2449,7,2449,0.0,0,0.0,100.0,5,100.0,d,0,0,0,0,27,10,20.00,5,20.00,87.0000,10,193.333333,10,193.3,87.0,17,0.020000,0,0.020000,75.00,10,75.00,24.00,24.00,g,24.00,590.0,590.0,kcal,590.0,2449.0,2449.0,kJ,2449.0,2449,kJ,45.000,45.000,g,45.000,0.0,g,100.0,100.0,3.0,3.0,3.0,17,17,20.00,g,0.05000,0.05000,g,0.05000,87.0000,87.0000,g,87.0000,0.020000,g,75.00,g,581.000,2413.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5400141039986,Carottes,https://images.openfoodfacts.org/images/produc...,Boni,Carottes,146,0,146,3.2,3,3.2,100.0,5,100.0,a,0,0,0,0,1,8,0.40,0,0.40,0.0000,0,0.000000,0,0.0,0.0,-7,0.040000,0,0.040000,6.50,1,6.50,6.70,6.70,g,6.7

In [5]:
df.columns.to_list()

['id',
 'name',
 'image_url',
 'brand',
 'categories',
 'energy',
 'energy_points',
 'energy_value',
 'fiber',
 'fiber_points',
 'fiber_value',
 'fruits_vegetables_nuts_colza_walnut_olive_oils',
 'fruits_vegetables_nuts_colza_walnut_olive_oils_points',
 'fruits_vegetables_nuts_colza_walnut_olive_oils_value',
 'grade',
 'is_beverage',
 'is_cheese',
 'is_fat',
 'is_water',
 'negative_points',
 'positive_points',
 'proteins',
 'proteins_points',
 'proteins_value',
 'saturated_fat',
 'saturated_fat_points',
 'saturated_fat_ratio',
 'saturated_fat_ratio_points',
 'saturated_fat_ratio_value',
 'saturated_fat_value',
 'score',
 'sodium',
 'sodium_points',
 'sodium_value',
 'sugars',
 'sugars_points',
 'sugars_value',
 'carbohydrates',
 'carbohydrates_100g',
 'carbohydrates_unit',
 'carbohydrates_value',
 'energy-kcal',
 'energy-kcal_100g',
 'energy-kcal_unit',
 'energy-kcal_value',
 'energy-kj',
 'energy-kj_100g',
 'energy-kj_unit',
 'energy-kj_value',
 'energy_100g',
 'energy_unit',
 'fat',


In [6]:
df[df["negative_points"] != (df["energy_points"] + df["saturated_fat_points"] + df["sugars_points"] + df["sodium_points"])]

,id,name,image_url,brand,categories,energy,energy_points,energy_value,fiber,fiber_points,fiber_value,fruits_vegetables_nuts_colza_walnut_olive_oils,fruits_vegetables_nuts_colza_walnut_olive_oils_points,fruits_vegetables_nuts_colza_walnut_olive_oils_value,grade,is_beverage,is_cheese,is_fat,is_water,negative_points,positive_points,proteins,proteins_points,proteins_value,saturated_fat,saturated_fat_points,saturated_fat_ratio,saturated_fat_ratio_points,saturated_fat_ratio_value,saturated_fat_value,score,sodium,sodium_points,sodium_value,sugars,sugars_points,sugars_value,carbohydrates,carbohydrates_100g,carbohydrates_unit,carbohydrates_value,energy-kcal,energy-kcal_100g,energy-kcal_unit,energy-kcal_value,energy-kj,energy-kj_100g,energy-kj_unit,energy-kj_value,energy_100g,energy_unit,fat,fat_100g,fat_unit,fat_value,fiber_100g,fiber_unit,fruits-vegetables-nuts-estimate-from-ingredients_100g,fruits-vegetables-nuts-estimate-from-ingredients_serving,nova-group,nova-group_100g,nova-group_serving,nutrition-score-fr,nutrition-score-fr_100g,proteins_100g,proteins_unit,salt,salt_100g,salt_unit,salt_value,saturated-fat,saturated-fat_100g,saturated-fat_unit,saturated-fat_value,sodium_100g,sodium_unit,sugars_100g,sugars_unit,energy-kcal_value_computed,energy-kj_value_computed,carbohydrates_serving,energy-kcal_serving,energy_serving,fat_serving,fiber_serving,proteins_serving,salt_serving,saturated-fat_serving,sodium_serving,sugars_serving,carbon-footprint-from-known-ingredients_100g,carbon-footprint-from-known-ingredients_product,carbon-footprint-from-known-ingredients_serving,energy-kj_serving


In [ ]:
df[df["id"] == 8480024002679]